In [1]:
# Alexander Farquharson, Sorting algorithm, notebook for creating integerise and sorting function, describing working, running examples

# Sorting algorithm:

## Working/thoughts:

  
### Input handling:

It is likely that the council receives data that is not expected (not a positive integer), the documentation describes the thinking behing what logic was created to handle these cases:
    
* float: e.g. 4.1, 4.15, 5.9. -> Would result in the input: 4, 4, 5
    * It is assumed that these refer to flats within a building, with the number before the . referring to the building no., the numbers after referring to the flat number within that building. As we only desire the building number, taking this assumption, only the first number is taken to produce an integer input.

        
* string: e.g. "1a", "pou23ujk","456:po","#7", "8,1", "9,a", "10:a" -> Would result in the input: 1, 8, 9, 10
     * It is assumed that this data is either wrong or refers to flats within a building (similar to the float example)
     * As a result, in order to get just the building number, if a number is first we take all digits upto when a number is no longer present, and replace the original string with that number
  
  
* duplicated integers:
    * No street should contain two buildings with the same number (would be confusing)
    * The input (after element processing) is deduped (e.g. if multiple flats were in the input) to contain unique integers only.
    
    
* Iterables: if an element is in a list, tuple, set, format:
    * iterate through these iterables to unpack their elements and then treat those elements as normal (repeat as many times as necessary)
        
        
* Other: if an element is any other data type like dict or bool:
    * Remove, can not think of any legitimate interpretation
  

In [1]:
import numpy as np
np.__version__

'1.20.3'

In [2]:
def integerise_input(input_array, output_array = np.array([], dtype = int)):
    '''Function to:
        Transform into an integer or remove all elements in the input list following a specific logic.
        Adds the manipulated elements to output_list.
    
    Inputs:
        Input_list: type - numpy.ndarray. List to apply function to.
        Output_list: type - numpy.ndarray. Default is [].
    
    Outputs:
        Deduped integerised flat numpy.ndarray
    '''

        
#     integerises elemnt and adds to output list
    for el in input_array:
        
#     necesssary to add in np.types
        if type(el) in (int, float, str, list, tuple, set, np.int32, np.float64, np.str_, np.ndarray):
            try:
                output_array = np.append(output_array, int(el))
    #     if element can not be integerised due to ValueError - string, use the below logic and add to output list if applicable
            except ValueError:
                pos = 0
                error_flag = 0
                while pos in list(range(len(el))) and error_flag == 0:
                    try:
                        int(el[pos])
                        pos+=1
                    except ValueError:
                        error_flag = 1

                try:
                    output_array = np.append(output_array, int(el[:pos]))
                except ValueError:
                    continue

    #     if typeerror - iterable, then iterate through that iterable using same logic above, adding elements to the output list
            except TypeError:
                output_array = integerise_input(input_array = el, output_array = output_array)

    return np.array(list(dict.fromkeys(output_array)))

#     For all but last step: time complexity will be a multiple of N (iterates over array once) (assuming N is linearly proportional to element string length as well as list length)
#     dict.fromkeys(list) - hashes every element (1*N), then adds to dictionary - looks up hash to see if key already in dictionary is real-time), so complexity of this step is N
#     Big(0) = N

In [3]:
arr = np.array([346,89,47,5,1,48,0])
integerise_input(arr)

array([346,  89,  47,   5,   1,  48,   0])

In [4]:
# Example
arr = np.array([1, 2, "3", "3.4", "4.1", "4.2", "5a", "5b", "isvb6aouo.fkpf", "7oadsufnfP", 8.1, 8.2, 9.097, [10,11,12], (13,14,15), {16,17,18, 18, 18}, True, {1:'two',3:'four'}])
integerise_input(arr)

C:\Users\farqu\AppData\Local\Temp/ipykernel_29812/449936631.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array([1, 2, "3", "3.4", "4.1", "4.2", "5a", "5b", "isvb6aouo.fkpf", "7oadsufnfP", 8.1, 8.2, 9.097, [10,11,12], (13,14,15), {16,17,18, 18, 18}, True, {1:'two',3:'four'}])


array([ 1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18])

### Sorting Efficiency:
    
* Option 1: 
    * Simple else if application of condition, then timsort the two lists:
    * Code as in answer:
        * Goes through list once applying "if even" condition N times.
        * Then sorts the equivalent of the full list once (using Timsort which has big(O) notation of NlogN)
        * Used python (instead of numpy) built-in sort algorithm as this has functionality for reverse order (hence the use of lists not arrays here)
        * Big(O) = N + NlogN ~= NlogN
            
            
* Option 2:
    * sorted([x for x in lst if x%2==1], reverse = True) + sorted([x for x in lst if x%2==0]) or
    * sorted(list(filter(lambda x: x%2 == 1,lst)), reverse = True) + sorted(list(filter(lambda x: x%2 == 0,lst)))
        * Tidier (one line) but applying x%2 condition through list twice - slightly less efficient than option 1.
        * In both cases, goes through list twice applying condition 2(N).
        * Then sorts the equivalent (actually sorts at most both half lists once) of the full list using timsort: NlogN.
        * Big(O) =2(N) + NlogN ~= NlogN - will be slightly less efficient than option 1.
            

In [5]:
def house_sort(arr, integerise = True):
    '''
    Function to:
        If specified integerise output.
        Then sorts inputed array such that odd numbers appear first in descending order, followed by the even numbers in ascending order
        
    Input:
        arr: type - array. Array to be sorted.
        integerise: type - bool. Default is true. If input array elements should be integerised
            
    Output:
        Sorted array of integers.
        '''

#     Integerise input
    if integerise:
        arr = integerise_input(input_array = arr) #x * N

#     Sorting algorithm (comments after line are contribution to Big 0 notation calculation at end)
    first_lst = [] #1
    second_lst = [] #1
    for el in arr: #N
        if el%2 ==1: #2N
            first_lst.append(el) #N
        else:
            second_lst.append(el) #N

    first_lst.sort(reverse = True) #1/2 (NlogN)
    second_lst.sort(reverse = False) #1/2(NlogN)
    return np.array(first_lst + second_lst) #1

    # Big(O) calculation for sorting part of algorithm
    # Big(O) = xN+1+1+N+2N+N+N+1/2(NlogN) + 1/2(NlogN) + 1 
    # Big(O) = NlogN

In [6]:
# Examples
arr = np.array([10,11,12, 1, 13,14,15, 5, 5, 16,17,18, 18, 18, 2,87,97], dtype = int)
house_sort(arr)

array([97, 87, 17, 15, 13, 11,  5,  1,  2, 10, 12, 14, 16, 18])

In [7]:
arr = np.array([1, 2, "3", "3.4", "4.1", "4.2", "5a", "5b", "isvb6aouo.fkpf", "7oadsufnfP", 8.1, 8.2, 9.097, [10,11,12], (13,14,15), {16,17,18, 18, 18}, True, {1:'two',3:'four'}])
house_sort(arr)

C:\Users\farqu\AppData\Local\Temp/ipykernel_29812/4083708640.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array([1, 2, "3", "3.4", "4.1", "4.2", "5a", "5b", "isvb6aouo.fkpf", "7oadsufnfP", 8.1, 8.2, 9.097, [10,11,12], (13,14,15), {16,17,18, 18, 18}, True, {1:'two',3:'four'}])


array([17, 15, 13, 11,  9,  7,  5,  3,  1,  2,  4,  8, 10, 12, 14, 16, 18])